In [1]:
from pytrends.request import TrendReq
import pandas as pd 
from datetime import datetime, timedelta
pytrend = TrendReq()

In [2]:
# Tratamento dos dados só para Lisboa (Interesse ao longo do tempo x Região)
def request_interest_over_time(indicator: str):
    first_date= '2014-01-01'
    first_date_datetime= datetime.strptime(first_date, '%Y-%m-%d')
    today = datetime.today()
    max_days = 5*365
    timeframe='today 5-y'
    pytrend.build_payload(kw_list=[indicator], timeframe=timeframe, geo='PT', cat=0)
    data = pytrend.interest_over_time()
    data = pd.DataFrame(data = data)
    data = data.reset_index(level = 0)
    data.drop(['isPartial'], axis=1, inplace=True)
    
    if (today-first_date_datetime).days >max_days:
        years_ago = today- timedelta(days=max_days)
        years_ago = datetime.strftime(years_ago, '%Y-%m-%d')
        timeframe= f'{first_date} {years_ago}'
        pytrend.build_payload(kw_list=[indicator], timeframe=timeframe, geo='PT', cat=0)
        data2 = pytrend.interest_over_time()
        data2 = pd.DataFrame(data = data2)
        data2 = data2.reset_index(level = 0)
        data2.drop(['isPartial'], axis=1, inplace=True)
        
        data = pd.concat([data, data2])
    return data

In [3]:
indicators= ['desemprego', 'emprego', 'net empregos', 'olx emprego',
       'subsidio desemprego', 'iefp', 'centro de emprego', 'linkedin',
       'ofertas emprego']

In [4]:
timeframe='today 5-y'

In [5]:
for indicator in indicators:
    data = request_interest_over_time(indicator)
    data.sort_values(by=['date'], inplace=True)
    if indicator==indicators[0]:
        final = data 
    else:
         final = pd.merge(final, data, how='outer', on='date')


In [6]:
final

,date,desemprego,emprego,net empregos,olx emprego,subsidio desemprego,iefp,centro de emprego,linkedin,ofertas emprego
0,2014-01-05,80,86,97,38,87,85,83,71,68
1,2014-01-12,74,90,90,34,72,79,88,70,75
2,2014-01-19,86,92,91,44,84,79,93,66,71
3,2014-01-26,79,89,95,52,73,78,80,78,65
4,2014-02-02,75,88,96,49,57,85,76,81,77
...,...,...,...,...,...,...,...,...,...,...
433,2022-04-24,32,64,68,42,29,59,32,75,60
434,2022-05-01,33,70,67,52,29,68,50,87,63
435,2022-05-08,29,69,74,46,30,68,39,83,60
436,2022-05-15,31,76,71,54,24,67,47,83,71


In [7]:
final.to_csv('Indicators.csv', index=False)